## 1. Library

In [ ]:
from selenium.webdriver import Chrome
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import quote
import time
import os
import pandas as pd
from tqdm import tqdm
import math
# 크롬 웹브라우저 실행
path = "chromedriver.exe"

## 2. 블로그 url 추출

In [ ]:
# 빈창 만들기
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36')

driver = webdriver.Chrome('chromedriver', options = options)
url_list = []
title=[]
name=[]
day=[]
content_list = ""
# 키워드 검색
text = "전기차 사용기"

for i in tqdm(range(1, math.ceil(523/7))):  # 블로그 건 수를 기준으로 블로그 내용을 읽어옴(한 페이지당 7개의 블로그 존재)
    url = 'https://section.blog.naver.com/Search/Post.naver?pageNo='+ str(i) +\
    '&rangeType=PERIOD&orderBy=sim&startDate=2016-01-01&endDate=2021-08-04&keyword=' + text
    driver.get(url)
    time.sleep(1)
 
    for j in range(1, 8): # 각 블로그 주소 저장(8개로!)
        url_titles = driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner'.format(j))
        url_title = url_titles.get_attribute('href')
        url_list.append(url_title)

        titles=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span'.format(j)).text
        title.append(titles)
        
        names=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > a > em'.format(j)).text
        name.append(names)

        days=driver.find_element_by_css_selector('#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.writer_info > span.date'.format(j)).text
        day.append(days)
                       
        
print("url 수집 끝, 해당 url 데이터 크롤링")

df_url = pd.DataFrame()
df_url['url'] = url_list
df_url['titles'] = title
df_url['names'] = name
df_url['days'] = day

## 3. 추출한 url로 본문 크롤링

In [ ]:
def make_iframe(url) :
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status() # 문제시 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml")
    
    src_url = "https://blog.naver.com/" + soup.iframe["src"]
    time.sleep(1)
    response = requests.get(src_url)
    time.sleep(1)
    logno = src_url.find('logNo=')
    red= src_url.find('&red')
    log_no =src_url[logno+6 : red]
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        source = '#post-view{}'.format(log_no)
        if soup.select_one(source) == [] :
            pass   
        else :
            title = soup.select_one(source).text.replace('\n','')
        # 기능, 번역보기 이후로 본문이 존재함 -> 정리
        if title.find('기능번역보기') != -1 :
            del_idx = title.find('기능번역보기')
            title = title[del_idx +6:]
        else :
            title = title
    print("-"*6,'진행중',"-"*6)
    return title

## 4. 파일 저장

In [ ]:
# 'utf-8' 인코딩으로 설정하는 함수
def encode_error(x) :
    x = x.encode('utf-8', 'replace').decode()
    return x

In [ ]:
# 인코딩 지정
df_url['리뷰'] = df_url.리뷰.apply(lambda x : encode_error(x))

In [ ]:
# xlsx 파일로 저장
writer = pd.ExcelWriter(r'전기차 사용기.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_url.to_excel(writer)
writer.close()